# Load Data

In [1]:
import pandas as pd
import numpy as np
import os, pickle
from tqdm import tqdm_notebook as tqdm

In [112]:
dataset_dir = '../ExtraSensory.per_uuid_features_labels'
label_dir = '../ExtraSensory.per_uuid_original_labels'
data = pd.DataFrame()
for filename in tqdm(os.listdir(dataset_dir)):
    key = filename[:filename.find('.')]
    csv = pd.read_csv(os.path.join(dataset_dir, filename), index_col=0)
    label = pd.read_csv(os.path.join(label_dir, key+'.original_labels.csv.gz'), index_col=0)
    csv = csv.join(label.drop('label_source', axis=1))
    csv = pd.concat([csv], keys=[key], names=['filename'])
    data = csv if data.empty else data.append(csv)

In [113]:
data.columns = pd.MultiIndex.from_tuples([tuple(i.split(':')) for i in data.columns], names=['1', '2', '3'])

In [114]:
labels_col = [i for i in data.columns if i[0].startswith('original_label')]
main_labels_col = [('main_label', i[1], i[2]) for i in labels_col[:7]]

In [115]:
data[labels_col[:7]].values.shape

(377346, 7)

In [116]:
df = pd.DataFrame(data[labels_col[:7]].values,\
                  columns=pd.MultiIndex.from_tuples(main_labels_col, names=['1', '2', '3']), \
                  index=data.index)

data = data.join(df)

In [120]:
data[('main_label', 'missing', np.nan)] = 1-data['main_label'].sum(axis=1)

In [122]:
data.shape

(377346, 401)

In [123]:
data.head()

1                                                       raw_acc            \
2                                               magnitude_stats             
3                                                          mean       std   
filename                             timestamp                              
11B5EC4D-4133-4289-B475-4E737182A406 1440000455        1.004012  0.040490   
                                     1440000515        1.002203  0.011955   
                                     1440000575        1.002147  0.005930   
                                     1440000635        1.001937  0.005161   
                                     1440000695        1.005683  0.066262   

1                                                                    \
2                                                                     
3                                                 moment3   moment4   
filename                             timestamp                        
11B5EC4D-4133-4289-B475-4E737182A406 1440000455  0.100981  0.178834   
                                     1440000515 -0.004731  0.025712   
                                     1440000575  0.006193  0.014327   
                                     1440000635 -0.007264  0.015998   
                                     1440000695  0.166331  0.291310   

1                                                                          \
2                                                                           
3                                               percentile25 percentile50   
filename                             timestamp                              
11B5EC4D-4133-4289-B475-4E737182A406 1440000455     1.000016     1.003875   
                                     1440000515     0.999446     1.002202   
                                     1440000575     0.999737     1.002325   
                                     1440000635     0.999730     1.001946   
                                     1440000695     0.999031     1.003126   

1                                                                           \
2                                                                            
3                                               percentile75 value_entropy   
filename                             timestamp                               
11B5EC4D-4133-4289-B475-4E737182A406 1440000455     1.006938      0.327604   
                                     1440000515     1.004927      1.209564   
                                     1440000575     1.004686      1.214844   
                                     1440000635     1.004220      0.919200   
                                     1440000695     1.008375      0.353075   

1                                                             \
2                                                              
3                                               time_entropy   
filename                             timestamp                 
11B5EC4D-4133-4289-B475-4E737182A406 1440000455     6.683913   
                                     1440000515     6.684541   
                                     1440000575     6.684594   
                                     1440000635     6.684598   
                                     1440000695     6.682873   

1                                                                    ...    \
2                                               magnitude_spectrum   ...     
3                                                 log_energy_band0   ...     
filename                             timestamp                       ...     
11B5EC4D-4133-4289-B475-4E737182A406 1440000455           5.043690   ...     
                                     1440000515           5.043621   ...     
                                     1440000575           5.043792   ...     
                                     1440000635           5.043255   ...     
                                     1440000695           5.036964  

# Train/test split

In [124]:
users = list(set(data.index.get_level_values(0)))
np.random.shuffle(users)
train_user_num = int(len(users)*0.8)
valid_user_num = int(len(users)*0.1)
test_user_num = len(users) - train_user_num - valid_user_num

In [125]:
train, valid = data.loc[users[:train_user_num]], data.loc[users[train_user_num:train_user_num+valid_user_num]]
test = data.loc[users[-test_user_num:]]

In [126]:
len(train), len(valid), len(test)

(287420, 44632, 45294)

In [127]:
pickle.dump((train, valid, test), open('dataset.pickle', 'wb'))